In [1]:
import numpy as np
import pandas as pd
#读取数据
a = pd.read_csv("/DATA7_DB7/data/fhong/datamining2020course/course1.csv")
a=np.asarray(a)
data=a[:,1:]
idn=a[:,0]

In [2]:
import random
#初始聚类中心
def int_center(data, k=5):
    index = random.sample(range(0,49999),5)
    centers=data[index,:]
    return centers


In [3]:
#根据中心分配簇
def devide(data, k, centers):

    clusters = []
    for index in range(k):
        clusters.append([])

    for i,x in enumerate(data):
        dis = np.sum((x-centers)**2, axis=1)
        idx=np.argmin(dis)
        clusters[idx].append(i)

    return clusters


In [4]:
#更新聚类中心
def update(data, k, clusters):
    centers = np.zeros((k, data.shape[1]))

    for idx in range(k):
        
        if len(clusters[idx]):

            tmp=np.array(clusters[idx])

            centers[idx] = np.mean(data[tmp,:], axis=0)

    return centers


In [5]:
#kmeans聚类
def classify(data, k, threshold):

    centers = int_center(data, k)
    loss = float("inf")
    m=1

    while loss > threshold :
        m=m+1
        
        clusters = devide(data, k, centers)
        old_centers = centers
        centers = update(data, k, clusters)
        loss = np.sum(np.abs(old_centers - centers))
        print(" loss=%f" % (loss))
        #不收敛重新初始化
        if m>5:
            m=1
            centers = int_center(data, k)
            loss = float("inf")
        #出现簇没有点重新初始化
        if loss < threshold:
            for i in range(k):
                if len(clusters[i])==0:
                    m=1
                    centers = int_center(data, k)
                    loss = float("inf")
                    break
    
    return centers, clusters


In [6]:
centers, clusters= classify(data=data,k=5,threshold=0.01)

 loss=109.844261
 loss=0.487476
 loss=0.340194
 loss=0.251572
 loss=0.222715
 loss=306.676628
 loss=94.227368
 loss=0.000000


In [7]:
#每个簇的大小
for i in range(5):
    print(len(clusters[i]))

10000
10000
10000
10000
10000


In [8]:
#计算每个簇的半径
for i in range(5):
    tmp=np.array(clusters[i])
    distance=np.sum((data[tmp,:]-centers[i])**2, axis=1)
    print(distance.shape)
    distance=np.max(distance)
    print(distance)

(10000,)
4.9381935446020995
(10000,)
5.551432076967877
(10000,)
4.1708935581031445
(10000,)
4.656891069425747
(10000,)
5.554687861387594


In [9]:
out= idn
#根据半径大小（升序）打标签
tmp=np.array(clusters[0])
out[tmp]=2
tmp=np.array(clusters[1])
out[tmp]=3
tmp=np.array(clusters[2])
out[tmp]=0
tmp=np.array(clusters[3])
out[tmp]=1
tmp=np.array(clusters[4])
out[tmp]=4

In [10]:
printarray=np.c_[idn,out]
print(printarray.astype(np.int))
print(printarray.shape)

[[0 0]
 [4 4]
 [4 4]
 ...
 [4 4]
 [3 3]
 [0 0]]
(50000, 2)


In [11]:
list=printarray.tolist()
name=['id','category']
result=pd.DataFrame(columns=name,data=list)
print(result)
result.to_csv('result.csv')

        id  category
0      0.0       0.0
1      4.0       4.0
2      4.0       4.0
3      1.0       1.0
4      4.0       4.0
...    ...       ...
49995  3.0       3.0
49996  4.0       4.0
49997  4.0       4.0
49998  3.0       3.0
49999  0.0       0.0

[50000 rows x 2 columns]
